In [8]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
from datetime import datetime, timedelta

# Get configuration data

Récupérer les données de
https://www.historique-meteo.net/france/ile-de-france/2020/01/  
Par jour https://www.historique-meteo.net/france/alsace/2020/01/01/

In [17]:
refs = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/region_meteo/region_meteo.csv")
refs.tail(3)

,region,url,Tempmax,Tempmin,Humidite,Dureejour
20,poitou-charentes,https://www.historique-meteo.net/france/poitou...,"//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/..."
21,rh-ne-alpes,https://www.historique-meteo.net/france/rh-ne-...,"//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/..."
22,provence-alpes-c-te-d-azur,https://www.historique-meteo.net/france/rh-ne-...,"//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/...","//*[@id=""content""]/div/div/div[1]/table/tbody/..."


# Get data from each page (per region and per day)

In [18]:
def getTemp(_val):
  try:
    myval = _val.replace('Â', '').replace('°', '')
    return int(myval)
  except:
    return np.nan

def getWet(_val):
  try:
    myval = _val.replace('%', '')
    return int(myval)
  except:
    return np.nan

def getSunTimeInSec(_val):
  try:
    vals = _val.split(':')
    return int(vals[2]) + int(vals[1]) * 60 +  int(vals[0]) * 60 * 60
  except:
    return np.nan

# Return the text value from an XPath
def getValueFromXPath(doc, _xpath):
  try:
    value = doc.xpath(_xpath)
    return value[0].text_content().strip()
  except:
    return "Error"

# Return the meteo data for 1 day / 1 region
def getRegionMeteofromDay(_index, _day):
  page = requests.get(refs['url'][_index] + _day)
  doc = lh.fromstring(page.content)
  tempmax = getValueFromXPath(doc, refs['Tempmax'][_index])
  tempmin = getValueFromXPath(doc, refs['Tempmin'][_index])
  humidite = getValueFromXPath(doc, refs['Humidite'][_index])
  dureejour = getValueFromXPath(doc, refs['Dureejour'][_index])
  return getTemp(tempmax), getTemp(tempmin), getWet(humidite), getSunTimeInSec(dureejour)

Test with one value

In [14]:
tmax, tmin, wet, timeduration = getRegionMeteofromDay(1, "2020/01/01/")
print (tmax, tmin, wet, timeduration)

12 10 89 31920


# Loop per day for all region (Create a CSV file for each region for all days)

In [23]:
# Counter date function
starting_date = datetime(2020, 1, 1)
today = datetime.now()

def formatDateForURL(_date):
  return (str(_date.year).zfill(4) + "/" + str(_date.month).zfill(2) + "/" + str(_date.day).zfill(2)) 

def addDate(_counteridx):
  return formatDateForURL(starting_date + timedelta(_counteridx))

#sending_date = formatDateForURL(today)
sending_date = '2020/12/01'
sstarting_date = formatDateForURL(starting_date)

Create one file per region (take some minutes to achieve)

In [24]:
for indexRegion in range (len(refs)):
  reg1 = pd.DataFrame()
  i = 0
  date = sstarting_date

  print("Start:", refs['region'][indexRegion])
  while date != sending_date:
    if not(i % 100):
      print(f"Date: {date} / index: {i}")
    date = addDate(i)
    tmax, tmin, wet, timeduration = getRegionMeteofromDay(indexRegion, date)
    reg1 = reg1.append([(refs['region'][indexRegion], date, tmax, tmin, wet, timeduration)])
    i = i + 1

  # Change columns names
  reg1.columns = ['region', 'date', 'tmax', 'tmin', 'wet', 'suntime']
  reg1.to_csv('/content/drive/MyDrive/Colab Notebooks/region_meteo/' + refs['region'][indexRegion] + '.csv')

Start: alsace
Date: 2020/01/01 / index: 0
Date: 2020/04/09 / index: 100
Date: 2020/07/18 / index: 200
Date: 2020/10/26 / index: 300
Start: aquitaine
Date: 2020/01/01 / index: 0
Date: 2020/04/09 / index: 100
Date: 2020/07/18 / index: 200
Date: 2020/10/26 / index: 300
Start: ardeche
Date: 2020/01/01 / index: 0
Date: 2020/04/09 / index: 100
Date: 2020/07/18 / index: 200
Date: 2020/10/26 / index: 300
Start: auvergne
Date: 2020/01/01 / index: 0
Date: 2020/04/09 / index: 100
Date: 2020/07/18 / index: 200
Date: 2020/10/26 / index: 300
Start: bourgogne
Date: 2020/01/01 / index: 0
Date: 2020/04/09 / index: 100
Date: 2020/07/18 / index: 200
Date: 2020/10/26 / index: 300
Start: bretagne
Date: 2020/01/01 / index: 0
Date: 2020/04/09 / index: 100
Date: 2020/07/18 / index: 200
Date: 2020/10/26 / index: 300
Start: centre
Date: 2020/01/01 / index: 0
Date: 2020/04/09 / index: 100
Date: 2020/07/18 / index: 200
Date: 2020/10/26 / index: 300
Start: champagne-ardenne
Date: 2020/01/01 / index: 0
Date: 2020/0

# Agregate the region files for temperatures (X: Region / Y: Date)

In [25]:
all_regions = pd.DataFrame()
for index, row in refs.iterrows():
  curRegionName = row['region']
  print(f"Region: {curRegionName}")
  # Managing columns
  curentregion = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/region_meteo/'+ curRegionName + '.csv')
  curentregion = curentregion.drop(['region','Unnamed: 0', 'wet', 'suntime'], axis=1)
  curentregion['T_' + curRegionName] = (curentregion['tmax'] + curentregion['tmin']) / 2
  curentregion = curentregion.drop(['tmax', 'tmin'], axis=1)
  
  # Merge & Add cols per region
  if (len(all_regions) == 0):
    all_regions = curentregion
  else:
    all_regions = all_regions.merge(curentregion, how='left', on='date')

Region: alsace
Region: aquitaine
Region: ardeche
Region: auvergne
Region: bourgogne
Region: bretagne
Region: centre
Region: champagne-ardenne
Region: corse
Region: franche-comte
Region: ile-de-re
Region: ile-de-france
Region: languedoc-roussillon
Region: limousin
Region: lorraine
Region: midi-pyrenees
Region: nord-pas-de-calais
Region: normandie
Region: pays-de-la-loire
Region: picardie
Region: poitou-charentes
Region: rh-ne-alpes
Region: provence-alpes-c-te-d-azur


In [26]:
all_regions = all_regions[all_regions['T_alsace'] > 0]
print(len(all_regions))

336


In [27]:
all_regions.to_csv('/content/drive/MyDrive/Colab Notebooks/region_meteo/' + 'all_regions_temperatures.csv')

# Concat region in one file (X: all data, Y Region + Date)

In [28]:
all_regions = pd.DataFrame()
for index, row in refs.iterrows():
  curRegionName = row['region']
  print(f"Region: {curRegionName}")
  # Managing columns
  curentregion = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/region_meteo/'+ curRegionName + '.csv')
  curentregion = curentregion.drop(['Unnamed: 0'], axis=1)
  curentregion['tavg'] = (curentregion['tmax'] + curentregion['tmin']) / 2
  
  # concat
  all_regions = pd.concat([all_regions, curentregion])

Region: alsace
Region: aquitaine
Region: ardeche
Region: auvergne
Region: bourgogne
Region: bretagne
Region: centre
Region: champagne-ardenne
Region: corse
Region: franche-comte
Region: ile-de-re
Region: ile-de-france
Region: languedoc-roussillon
Region: limousin
Region: lorraine
Region: midi-pyrenees
Region: nord-pas-de-calais
Region: normandie
Region: pays-de-la-loire
Region: picardie
Region: poitou-charentes
Region: rh-ne-alpes
Region: provence-alpes-c-te-d-azur


In [29]:
len(all_regions)

7728

In [30]:
all_regions.to_csv('/content/drive/MyDrive/Colab Notebooks/region_meteo/' + 'all_regions.csv', index=False)

# New France Region mapping

The covid data collected refers to old regions !

In [31]:
all_regions = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/region_meteo/all_regions.csv')

reg_target = ['Île-de-France', 'Nouvelle-Aquitaine', 'Auvergne-Rhône-Alpes',
       'Bourgogne-Franche-Comté', 'Hauts-de-France', 'Grand Est',
       'Guadeloupe', 'Martinique', 'Guyane', 'La Réunion', 'Mayotte',
       'Centre-Val de Loire', 'Normandie', 'Pays de la Loire', 'Bretagne',
       'Occitanie', "Provence-Alpes-Côte d'Azur", 'Corse']

all_regions.head(3)

,region,date,tmax,tmin,wet,suntime,tavg
0,alsace,2020/01/01,6,1.0,NaN,NaN,3.5
1,alsace,2020/01/02,5,2.0,80.0,30180.0,3.5
2,alsace,2020/01/03,7,3.0,87.0,30240.0,5.0


In [32]:
all_regions['region'] = all_regions['region'].map({'ile-de-france' : 'Île-de-France',
                                                   'limousin' : 'Nouvelle-Aquitaine',
                                                   'aquitaine' : 'Nouvelle-Aquitaine',
                                                   'poitou-charentes' : 'Nouvelle-Aquitaine',
                                                   '' : 'Auvergne-Rhône-Alpes',
                                                   'bourgogne' : 'Bourgogne-Franche-Comté',
                                                   'franche-comte' : 'Bourgogne-Franche-Comté',
                                                   'nord-pas-de-calais' : 'Hauts-de-France',
                                                   'picardie' : 'Hauts-de-France',
                                                   'alsace' : 'Grand Est',
                                                   'lorraine' : 'Grand Est',
                                                   '--' : 'Guadeloupe',
                                                   '--' : 'Martinique',
                                                   '--' : 'Guyane',
                                                   '--' : 'La Réunion',
                                                   '--' : 'Mayotte',
                                                   'centre' : 'Centre-Val de Loire',
                                                   'normandie' : 'Normandie',
                                                   'pays-de-la-loire' : 'Pays de la Loire',
                                                   'bretagne' : 'Bretagne',
                                                   'midi-pyrenees' : 'Occitanie',
                                                   'languedoc-roussillon' : 'Occitanie',
                                                   'provence-alpes-c-te-d-azur' : "Provence-Alpes-Côte d'Azur",
                                                   'corse' : 'Corse'
                                                  })

In [33]:
all_regions['region'].value_counts()

Nouvelle-Aquitaine            1008
Bourgogne-Franche-Comté        672
Grand Est                      672
Occitanie                      672
Hauts-de-France                672
Bretagne                       336
Centre-Val de Loire            336
Provence-Alpes-Côte d'Azur     336
Pays de la Loire               336
Normandie                      336
Corse                          336
Île-de-France                  336
Name: region, dtype: int64

In [34]:
all_regions.head()

,region,date,tmax,tmin,wet,suntime,tavg
0,Grand Est,2020/01/01,6,1.0,NaN,NaN,3.5
1,Grand Est,2020/01/02,5,2.0,80.0,30180.0,3.5
2,Grand Est,2020/01/03,7,3.0,87.0,30240.0,5.0
3,Grand Est,2020/01/04,6,4.0,85.0,30300.0,5.0
4,Grand Est,2020/01/05,7,3.0,84.0,30420.0,5.0


## Agregate on the regions (same date)

In [35]:
all_regions = all_regions.groupby(['region', 'date'], dropna=True).mean()
all_regions

tmax  tmin   wet  suntime  tavg
region                  date                                       
Bourgogne-Franche-Comté 2020/01/01   4.5   1.0   NaN      NaN  2.75
                        2020/01/02   5.5   2.0  87.5  30870.0  3.75
                        2020/01/03   7.0   4.0  91.5  30930.0  5.50
                        2020/01/04   6.5   5.0  90.5  30990.0  5.75
                        2020/01/05   5.5   2.5  88.5  31080.0  4.00
...                                  ...   ...   ...      ...   ...
Île-de-France           2020/11/27  11.0   8.0   NaN      NaN  9.50
                        2020/11/28  12.0   7.0   NaN      NaN  9.50
                        2020/11/29  10.0   6.0  72.0  31020.0  8.00
                        2020/11/30   8.0   3.0  74.0  30900.0  5.50
                        2020/12/01   9.0   8.0  76.0  30780.0  8.50

[4032 rows x 5 columns]

In [36]:
all_regions.to_csv('/content/drive/MyDrive/Colab Notebooks/region_meteo/' + 'all_newregions.csv')